# 03 - API Integration Test

Use this notebook to run simple integration tests against the local FastAPI backend. The notebook posts `data/sample_sales.csv` to `/predict` and verifies the response shape.

Make sure the backend is running (e.g., `uvicorn app.main:app --reload --port 8000`) or run via Docker Compose.

In [ ]:
import requests
import json
import pandas as pd
from pathlib import Path

BACKEND_URL = 'http://localhost:8000'  # change if needed
csv_path = Path('..') / 'data' / 'sample_sales.csv'
assert csv_path.exists(), f'Sample CSV not found at {csv_path}'

files = {'file': ('sample_sales.csv', open(csv_path, 'rb'), 'text/csv')}

print('Posting to', BACKEND_URL + '/predict')
resp = requests.post(BACKEND_URL + '/predict', files=files, timeout=120)
print('Status code:', resp.status_code)
if resp.status_code == 200:
    data = resp.json()
    print('Historical points:', len(data.get('historical_data', {}).get('timestamp', [])))
    print('Forecast points:', len(data.get('forecast_data', {}).get('timestamp', [])))
    # Basic assertion checks
    assert 'historical_data' in data
    assert 'forecast_data' in data
    assert 'best_model' in data
    assert 'metrics' in data
    print('Success: API returned expected fields.')
else:
    print('Error response text:', resp.text)
    resp.raise_for_status()

## Notes
- If the call times out or connection is refused, make sure the backend is running and `BACKEND_URL` is correct.
- This notebook is intended for manual integration checks — for automated CI you should write pytest-based integration tests.